# Genobear exploration notebook



In [5]:
import os
import polars as pl
import polars_bio as pb
from pathlib import Path
from pycomfort import files
import polars_bio as pb
# Configure Polars to show more rows and columns
pl.Config.set_tbl_rows(-1)  # Show all rows (or use a large number like 1000)
pl.Config.set_tbl_cols(-1)  # Show all columns (or use a large number like 50)
pl.Config.set_tbl_width_chars(1000)  # Increase table width to prevent column truncation
pl.Config.set_fmt_str_lengths(1000)  # Show longer string values without truncation







polars.config.Config

In [6]:
from pycomfort.logging import to_nice_stdout, to_nice_file
to_nice_stdout()

In [7]:
from platformdirs import user_cache_dir
from pycomfort.files import tprint
cache_dir = Path(user_cache_dir(appname="genobear"))
print(cache_dir.absolute().resolve())
tprint(cache_dir)

/home/antonkulaga/.cache/genobear
genobear
	ensembl_vcf
		homo_sapiens-chr1.vcf.gz
		homo_sapiens-chr5.vcf.gz
		20f9321d22b6bf386b0a23d2499243e195aff8e21d1654a53c0da8019c8a8b7a
		homo_sapiens-chr6.vcf.gz
		0416f6134bd5c84286ebd4fffe845a4d6768e6f86f069a0773eff784aa4c084d
		765b8fa0647d27df66fbfa8c859f77d0c565bbb9e7ab15760b7e0e0e84a59638
		dca203f561595c9b659bd8b85a57c4b23e1ff6d02bced4cdc786b77fd31c299c
		2e23a05fd1d65acd59d1d9342e201e0f96aef41d030ba274858a733a3b89af45
		955092064d2abb285773b06afeba9f4c5433ef6c7e44d484c8a2d8ec76f5a89c
		8b12c9270181eff64730438982c27e1abf94bc8cc306d81afecba3bcdb3f2756
		b29bc733eeeefd2f16b04d3b2b388f708d012d2da8b2e8548843c1a7fe09e6dd
		3e6b27d87ef73d02741a7a31dfadc5500c625cf849e15b27fb2eddf0e593a4d7
		homo_sapiens-chr9.vcf.gz
		a98b26f48b8ae5cff08d83198727e25ddf71e9f74999d8b7cc68ae23fad793d1
		b6f04df9d2463af48af3ed7a076f31c747bfb76d926eea0e99f8e5fb99aee2de
		homo_sapiens-chr10.vcf.gz
		a1e660f1b6f8881723856fb15da4edc90431fc4da7788f10ace8a998b8c4f1df
		ho

In [16]:
from pycomfort.files import with_ext


genobear = Path.home() / ".cache" / "genobear"
ensembl = genobear / "ensembl_variations"
with_ext(ensembl, "parquet").to_list()

[PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr2.parquet'),
 PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr9.parquet'),
 PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr17.parquet'),
 PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr21.parquet'),
 PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr8.parquet'),
 PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chrY.parquet'),
 PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr20.parquet'),
 PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr19.parquet'),
 PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr4.parquet'),
 PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr14.parquet'),
 PosixPath('/home/antonkulaga/.cache/genobear/ensembl_v

In [ ]:
from pycomfort.files import children


first = children(ensembl).filter(lambda f: "chr1.parquet" in f.name)

[PosixPath('/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr1.parquet')]

Ensembl test
============

In [ ]:
import polars_bio as pb
chr1 = pb.scan_vcf("/home/antonkulaga/.cache/genobear/ensembl_variations")
chr1.collect().count()


thread '<unnamed>' panicked at /root/.cargo/git/checkouts/datafusion-bio-formats-f4a7f32bff6627c2/f62b38b/datafusion/bio-format-core/src/object_storage.rs:129:64:
called `Result::unwrap()` on an `Err` value: Os { code: 2, kind: NotFound, message: "No such file or directory" }
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: called `Result::unwrap()` on an `Err` value: Os { code: 2, kind: NotFound, message: "No such file or directory" }

# ClinVar Data Processing

## ClinVarDownloader Class Overview

The `ClinVarDownloader` class is a specialized downloader for ClinVar VCF files that inherits from `MultiVCFDownloader`. This inheritance provides several powerful capabilities:

### Inheritance Hierarchy
- **BaseModel** (Pydantic) - Provides data validation and serialization
- **ParquetPostprocessorMixin** - Adds variant splitting and parquet processing capabilities  
- **MultiVCFDownloader** - Handles concurrent downloads, caching, and parquet conversion
- **ClinVarDownloader** - Adds ClinVar-specific configuration and URL handling

### Key Features from MultiVCFDownloader
- **Automatic parquet conversion**: VCF files are converted to parquet for faster processing
- **Variant splitting**: Handles multi-allelic variants by splitting them into separate rows
- **Concurrent downloads**: Uses ThreadPoolExecutor for parallel file downloads
- **Intelligent caching**: Uses pooch for robust file caching and hash verification
- **LazyFrame integration**: Returns polars LazyFrames for memory-efficient data processing

### ClinVar-Specific Configuration
- **Assembly support**: Supports both GRCh37 and GRCh38 assemblies
- **Automatic URL construction**: Builds NCBI FTP URLs based on assembly version
- **Standardized file structure**: Uses consistent naming for VCF, index, and hash files
- **Cache organization**: Files are cached in `clinvar/{assembly}/` subdirectories

### Usage Pattern
The class automatically handles the complex URL mapping required by MultiVCFDownloader, converting single ClinVar file references into the multi-file format expected by the parent class.

In [1]:
df = pl.scan_parquet("/home/antonkulaga/.cache/genobear/ensembl_vcf/homo_sapiens-chr12.parquet")
df.head().collect()

NameError: name 'pl' is not defined

In [22]:
pl.scan_parquet("/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr4.vcf.parquet").head().collect()

chrom,start,end,id,ref,alt,qual,filter
str,u32,u32,str,str,str,f64,str
"""4""",10000,10000,"""rs1560073237""","""N""","""NG""",null,""""""
"""4""",10001,10001,"""rs1581341342""","""A""","""C""",null,""""""
"""4""",10002,10002,"""rs1560073246""","""C""","""CC""",null,""""""
"""4""",10002,10002,"""rs1581341346""","""C""","""A""",null,""""""
"""4""",10004,10004,"""rs1581341351""","""C""","""T""",null,""""""


In [21]:
import genobear as gb
gb.read_vcf_file("/home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr1.vcf.gz").head().collect()

ffa61ecb-8961-44e9-b0d9-a4fd320dda6c
└── read_vcf_file/1 ⇒ started 2025-09-03 16:14:01Z
    ├── file_path: /home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr1.vcf.gz
    └── save_parquet: auto

ffa61ecb-8961-44e9-b0d9-a4fd320dda6c
└── <unnamed>
    └── info/2 2025-09-03 16:14:01Z
        ├── parquet_path: /home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr1.vcf.parquet
        └── step: reading_vcf

ffa61ecb-8961-44e9-b0d9-a4fd320dda6c
└── <unnamed>
    └── get_info_fields/3/1 ⇒ started 2025-09-03 16:14:01Z
        └── vcf_path: /home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr1.vcf.gz

ffa61ecb-8961-44e9-b0d9-a4fd320dda6c
└── <unnamed>
    └── <unnamed>
        └── info/3/2 2025-09-03 16:14:01Z
            ├── count: 0
            ├── fields: 
            └── step: info_fields_extracted

ffa61ecb-8961-44e9-b0d9-a4fd320dda6c
└── <unnamed>
    └── <unnamed>
        └── get_info_fields/3/3 ⇒ succeeded 2025-09-03 16:14:01Z

ffa61ecb

86821100rows [00:46, 1875726.67rows/s]


ffa61ecb-8961-44e9-b0d9-a4fd320dda6c
└── <unnamed>
    └── <unnamed>
        └── info/5/2 2025-09-03 16:14:58Z
            ├── parquet_path: /home/antonkulaga/.cache/genobear/ensembl_variations/homo_sapiens-chr1.vcf.parquet
            └── step: parquet_saved

ffa61ecb-8961-44e9-b0d9-a4fd320dda6c
└── <unnamed>
    └── <unnamed>
        └── save_parquet/5/3 ⇒ succeeded 2025-09-03 16:14:58Z

ffa61ecb-8961-44e9-b0d9-a4fd320dda6c
└── <unnamed>
    └── read_vcf_file/6 ⇒ succeeded 2025-09-03 16:14:58Z



chrom,start,end,id,ref,alt,qual,filter
str,u32,u32,str,str,str,f64,str
"""1""",10001,10001,"""rs1570391677""","""T""","""A|C""",null,""""""
"""1""",10002,10002,"""rs1570391692""","""A""","""C""",null,""""""
"""1""",10003,10003,"""rs1570391694""","""A""","""C""",null,""""""
"""1""",10007,10007,"""rs1639538116""","""T""","""C|G""",null,""""""
"""1""",10008,10008,"""rs1570391698""","""A""","""C|G|T""",null,""""""


In [8]:
from genobear.downloaders.clinvar_downloader import ClinVarDownloader
from genobear.downloaders.multi_vcf_downloader import DownloadResult

# Initialize ClinVar downloader for GRCh38 assembly
clinvar_downloader = ClinVarDownloader(assembly='GRCh38')

# Download and convert ClinVar VCF data to parquet
download_results = clinvar_downloader.download_all()

# Get the download result for ClinVar
clinvar_result: DownloadResult = download_results['clinvar']

print(f"ClinVar VCF downloaded to: {clinvar_result.vcf}")
print(f"ClinVar index downloaded to: {clinvar_result.index}")
print(f"ClinVar parquet converted to: {clinvar_result.parquet}")

# Access the lazy frame for efficient processing
clinvar_lazy = clinvar_result.get_lazy_frame()

# Show basic info about the ClinVar dataset
print(f"ClinVar data shape: {clinvar_lazy.collect().shape}")
clinvar_lazy.head().collect()

1d27cb5a-9f8e-48d9-bb85-e5c564adf6e4
└── multi_vcf_download/1 ⇒ started 2025-09-03 15:00:38Z
    └── num_files: 1

1d27cb5a-9f8e-48d9-bb85-e5c564adf6e4
└── <unnamed>
    └── parallel_downloads/2/1 ⇒ started 2025-09-03 15:00:38Z
        ├── tasks: 2
        └── workers: 8

6f9791a2-1d31-4679-9559-7e8c93ce4d68
└── download_single_file/1 ⇒ started 2025-09-03 15:00:38Z
    ├── file_type: vcf
    ├── filename: clinvar.vcf.gz
    ├── identifier: clinvar
    └── url: https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz

dc7112dd-1686-454a-93ac-8491ab82ea38
└── download_single_file/1 ⇒ started 2025-09-03 15:00:38Z
    ├── file_type: index
    ├── filename: clinvar.vcf.gz.tbi
    ├── identifier: clinvar
    └── url: https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz.tbi



100%|████████████████████████████████████████| 564k/564k [00:00<00:00, 908MB/s]
SHA256 hash of downloaded file: 62030753ecc55905890b1c44c64918c319a5e09e7b20c45853c42d15c1878fb5
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


dc7112dd-1686-454a-93ac-8491ab82ea38
└── <unnamed>
    └── info/2 2025-09-03 15:00:39Z
        ├── path: /home/antonkulaga/.cache/genobear/clinvar/GRCh38/clinvar.vcf.gz.tbi
        └── status: completed

dc7112dd-1686-454a-93ac-8491ab82ea38
└── <unnamed>
    └── download_single_file/3 ⇒ succeeded 2025-09-03 15:00:39Z



100%|████████████████████████████████████████| 169M/169M [00:00<00:00, 307GB/s]
SHA256 hash of downloaded file: 9a60cd19904af97367845ae800a4c9fc1fe029f9e1952a9868aacaf41da5d6e7
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


6f9791a2-1d31-4679-9559-7e8c93ce4d68
└── <unnamed>
    └── info/2 2025-09-03 15:00:55Z
        ├── path: /home/antonkulaga/.cache/genobear/clinvar/GRCh38/clinvar.vcf.gz
        └── status: completed

6f9791a2-1d31-4679-9559-7e8c93ce4d68
└── <unnamed>
    └── download_single_file/3 ⇒ succeeded 2025-09-03 15:00:55Z

1d27cb5a-9f8e-48d9-bb85-e5c564adf6e4
└── <unnamed>
    └── <unnamed>
        └── parallel_downloads/2/2 ⇒ succeeded 2025-09-03 15:00:55Z

1d27cb5a-9f8e-48d9-bb85-e5c564adf6e4
└── <unnamed>
    └── convert_all_to_parquet/3/1 ⇒ started 2025-09-03 15:00:55Z
        └── num_files: 1

1d27cb5a-9f8e-48d9-bb85-e5c564adf6e4
└── <unnamed>
    └── <unnamed>
        └── convert_single_vcf_to_parquet/3/2/1 ⇒ started 2025-09-03 15:00:55Z
            ├── identifier: clinvar
            └── vcf_path: /home/antonkulaga/.cache/genobear/clinvar/GRCh38/clinvar.vcf.gz

1d27cb5a-9f8e-48d9-bb85-e5c564adf6e4
└── <unnamed>
    └── <unnamed>
        └── <unnamed>
            └── vcf_to_parquet/3/2/2/

/home/antonkulaga/sources/genobear/src/genobear/io.py:81: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if hasattr(df, 'columns'):
/home/antonkulaga/sources/genobear/src/genobear/io.py:83: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  info_fields = [col[5:] for col in df.columns if col.startswith('info.')]
3678878rows [00:02, 1770907.65rows/s]


1d27cb5a-9f8e-48d9-bb85-e5c564adf6e4
└── <unnamed>
    └── <unnamed>
        └── <unnamed>
            └── <unnamed>
                └── <unnamed>
                    └── <unnamed>
                        └── info/3/2/2/3/5/2 2025-09-03 15:00:58Z
                            ├── parquet_path: /home/antonkulaga/.cache/genobear/clinvar/GRCh38/clinvar.vcf.parquet
                            └── step: parquet_saved

1d27cb5a-9f8e-48d9-bb85-e5c564adf6e4
└── <unnamed>
    └── <unnamed>
        └── <unnamed>
            └── <unnamed>
                └── <unnamed>
                    └── <unnamed>
                        └── save_parquet/3/2/2/3/5/3 ⇒ succeeded 2025-09-03 15:00:58Z

1d27cb5a-9f8e-48d9-bb85-e5c564adf6e4
└── <unnamed>
    └── <unnamed>
        └── <unnamed>
            └── <unnamed>
                └── <unnamed>
                    └── read_vcf_file/3/2/2/3/6 ⇒ succeeded 2025-09-03 15:00:58Z

1d27cb5a-9f8e-48d9-bb85-e5c564adf6e4
└── <unnamed>
    └── <unnamed>
        └── <unname

chrom,start,end,id,ref,alt,qual,filter
str,u32,u32,str,str,str,f64,str
"""1""",66926,66927,"""3385321""","""AG""","""A""",null,""""""
"""1""",69134,69134,"""2205837""","""A""","""G""",null,""""""
"""1""",69308,69308,"""3925305""","""A""","""G""",null,""""""
"""1""",69314,69314,"""3205580""","""T""","""G""",null,""""""
"""1""",69404,69404,"""3925306""","""T""","""C""",null,""""""


In [16]:
pl.scan_parquet(clinvar_result.parquet).head().collect()

chrom,start,end,id,ref,alt,qual,filter
str,u32,u32,str,str,str,f64,str
"""1""",66926,66927,"""3385321""","""AG""","""A""",null,""""""
"""1""",69134,69134,"""2205837""","""A""","""G""",null,""""""
"""1""",69308,69308,"""3925305""","""A""","""G""",null,""""""
"""1""",69314,69314,"""3205580""","""T""","""G""",null,""""""
"""1""",69404,69404,"""3925306""","""T""","""C""",null,""""""


In [17]:
pb.describe_vcf(str(clinvar_result.vcf))

name,type,description
str,str,str
"""AF_ESP""","""Float""","""allele frequencies from GO-ESP"""
"""AF_EXAC""","""Float""","""allele frequencies from ExAC"""
"""AF_TGP""","""Float""","""allele frequencies from TGP"""
"""ALLELEID""","""Integer""","""the ClinVar Allele ID"""
"""CLNDN""","""String""","""ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB"""
"""CLNDNINCL""","""String""","""For included Variant : ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB"""
"""CLNDISDB""","""String""","""Tag-value pairs of disease database name and identifier submitted for germline classifications, e.g. OMIM:NNNNNN"""
"""CLNDISDBINCL""","""String""","""For included Variant: Tag-value pairs of disease database name and identifier for germline classifications, e.g. OMIM:NNNNNN"""
"""CLNHGVS""","""String""","""Top-level (primary assembly, alt, or patch) HGVS expression."""


In [10]:
clinvar_lazy.count().collect()

chrom,start,end,id,ref,alt,qual,filter
u32,u32,u32,u32,u32,u32,u32,u32
3678878,3678878,3678878,3678878,3678878,3678878,0,3678878


In [ ]:
# Get ClinVar INFO fields
from genobear import get_info_fields
get_info_fields(str(clinvar_result.vcf))

AttributeError: module 'genobear' has no attribute 'get_info_fields'

In [ ]:
# Show some ClinVar data with important INFO fields
from genobear.io import read_vcf_file

clinvar_with_info = read_vcf_file(
    clinvar_result.vcf,
    streaming=True,
    info_fields=["CLNSIG", "CLNDN", "CLNHGVS", "CLNVC", "CLNREVSTAT"]
)

print("ClinVar columns:", clinvar_with_info.collect_schema().names())
clinvar_with_info.head(10).collect()

AttributeError: module 'genobear' has no attribute 'get_info_fields'

In [ ]:
# Count variants by clinical significance
clinvar_with_info.group_by("CLNSIG").agg(
    pl.count().alias("variant_count")
).sort("variant_count", descending=True).collect()

AttributeError: module 'genobear' has no attribute 'get_info_fields'

In [ ]:
# Use the ClinVar data for further analysis
# This replaces the old manual file loading approach
print(f"Total ClinVar variants: {clinvar_lazy.count().collect()}")
print(f"ClinVar VCF file: {clinvar_result.vcf}")
print(f"ClinVar parquet file: {clinvar_result.parquet}")

# Show a sample of the data
clinvar_lazy.head().collect()

chrom,start,end,id,ref,alt,qual,filter,af_esp,af_exac,af_tgp,alleleid,clndn,clndnincl,clndisdb,clndisdbincl,clnhgvs,clnrevstat,clnsig,clnsigconf,clnsigincl,clnsigscv,clnvc,clnvcso,clnvi,dbvarid,geneinfo,mc,oncdn,oncdnincl,oncdisdb,oncdisdbincl,onc,oncincl,oncrevstat,oncscv,oncconf,origin,rs,scidn,scidnincl,scidisdb,scidisdbincl,scirevstat,sci,sciincl,sciscv
str,u32,u32,str,str,str,f64,str,f32,f32,f32,i32,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],str,str,list[str],list[str],str,list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str],list[str]
"""1""",66926,66927,"""3385321""","""AG""","""A""",0.0,"""""",null,null,null,3544463,"[""Retinitis_pigmentosa""]",null,"[""Human_Phenotype_Ontology:HP:0000547"", ""MONDO:MONDO:0019200"", … ""Orphanet:791""]",null,"[""NC_000001.11:g.66927del""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV005419006""]","""Deletion""","""SO:0000159""",null,null,"""OR4F5:79501""","[""SO:0001627|intron_variant""]",null,null,null,null,null,null,null,null,null,"[""0""]",null,null,null,null,null,null,null,null,null
"""1""",69134,69134,"""2205837""","""A""","""G""",0.0,"""""",null,null,null,2193183,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69134A>G""]","[""criteria_provided"", ""_single_submitter""]","[""Likely_benign""]",null,null,"[""SCV003526545""]","""single_nucleotide_variant""","""SO:0001483""","[""ClinGen:CA502008""]",null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]","[""781394307""]",null,null,null,null,null,null,null,null
"""1""",69308,69308,"""3925305""","""A""","""G""",0.0,"""""",null,null,null,4039319,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69308A>G""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV006120601""]","""single_nucleotide_variant""","""SO:0001483""",null,null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]",null,null,null,null,null,null,null,null,null
"""1""",69314,69314,"""3205580""","""T""","""G""",0.0,"""""",null,null,null,3374047,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69314T>G""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV004995495""]","""single_nucleotide_variant""","""SO:0001483""","[""ClinGen:CA338197388""]",null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]","[""2521653848""]",null,null,null,null,null,null,null,null
"""1""",69404,69404,"""3925306""","""T""","""C""",0.0,"""""",null,null,null,4039320,"[""not_specified""]",null,"[""MedGen:CN169374""]",null,"[""NC_000001.11:g.69404T>C""]","[""criteria_provided"", ""_single_submitter""]","[""Uncertain_significance""]",null,null,"[""SCV006120602""]","""single_nucleotide_variant""","""SO:0001483""",null,null,"""OR4F5:79501""","[""SO:0001583|missense_variant""]",null,null,null,null,null,null,null,null,null,"[""1""]",null,null,null,null,null,null,null,null,null


Load Genomic Sample
===================

In [ ]:
from genobear.io import read_vcf_file
from pathlib import Path

data_folder = current_folder / "data"
test_folder = data_folder / "tests"

assert test_folder.exists(), f"Test folder {test_folder} does not exist"


In [ ]:
sample_vcf = test_folder / "antonkulaga.vcf"
sample = read_vcf_file(sample_vcf, info_fields=[], streaming=False)
sample.head().collect()

chrom,start,end,id,ref,alt,qual,filter
str,u32,u32,str,str,str,f64,str
"""1""",10009,10009,"""""","""A""","""AC""",0.0,"""RefCall"""
"""1""",10015,10015,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10021,10021,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10027,10027,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10033,10033,"""""","""A""","""G""",0.0,"""RefCall"""


## Annotation with clinvar

In [ ]:
sample.columns

/tmp/ipykernel_178433/374950290.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  sample.columns


['chrom', 'start', 'end', 'id', 'ref', 'alt', 'qual', 'filter']

In [ ]:
sample.join(clinvar_lazy, on=["chrom", "start", "end"], how="left").collect()

In [ ]:
pb.describe_vcf(str(clinvar_vcf))

name,type,description
str,str,str
"""AF_ESP""","""Float""","""allele frequencies from GO-ESP"""
"""AF_EXAC""","""Float""","""allele frequencies from ExAC"""
"""AF_TGP""","""Float""","""allele frequencies from TGP"""
"""ALLELEID""","""Integer""","""the ClinVar Allele ID"""
"""CLNDN""","""String""","""ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB"""
"""CLNDNINCL""","""String""","""For included Variant : ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB"""
"""CLNDISDB""","""String""","""Tag-value pairs of disease database name and identifier submitted for germline classifications, e.g. OMIM:NNNNNN"""
"""CLNDISDBINCL""","""String""","""For included Variant: Tag-value pairs of disease database name and identifier for germline classifications, e.g. OMIM:NNNNNN"""
"""CLNHGVS""","""String""","""Top-level (primary assembly, alt, or patch) HGVS expression."""


In [ ]:
# Test the function with both regular and gzipped VCF files
sample_small = read_vcf_file(sample_small_vcf)

NameError: name 'sample_small_vcf' is not defined

In [ ]:
data_folder = Path("../data").absolute().resolve()
test_folder = data_folder / "test"

files.tprint(test_folder)


test
	longevity_snps_1000genom.vcf.gz
	antonkulaga.vcf
	antku_small.vcf
	longevity_snps_1000genom.vcf


## Samples

Loading samples

In [ ]:
sample_antku = pb.scan_vcf(str(test_folder / "antku_small.vcf"))
sample_antonkulaga = pb.scan_vcf(str(test_folder / "antonkulaga.vcf"))
sample_antku.head().collect()

chrom,start,end,id,ref,alt,qual,filter
str,u32,u32,str,str,str,f64,str
"""1""",10009,10009,"""""","""A""","""AC""",0.0,"""RefCall"""
"""1""",10015,10015,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10021,10021,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10027,10027,"""""","""A""","""G""",0.0,"""RefCall"""
"""1""",10033,10033,"""""","""A""","""G""",0.0,"""RefCall"""


In [ ]:
sample_antku.count().collect()

793rows [00:00, 750300.72rows/s]


chrom,start,end,id,ref,alt,qual,filter
u32,u32,u32,u32,u32,u32,u32,u32
793,793,793,793,793,793,793,793


In [ ]:
sample_antonkulaga.count().collect()

6079744rows [00:04, 1414880.55rows/s]


chrom,start,end,id,ref,alt,qual,filter
u32,u32,u32,u32,u32,u32,u32,u32
6079744,6079744,6079744,6079744,6079744,6079744,6079744,6079744


In [ ]:
# Let's examine the ALT column structure first
# Look at some data to understand the pipe-separated ALT values
sample_data = sample_antku.head(50).collect()
print("Sample data columns:", sample_data.columns)
print("\nFirst few rows:")
print(sample_data.select(["chrom", "pos", "ref", "alt"]).head(10))

# Check for pipe-separated ALT alleles
alt_with_pipes = sample_data.filter(pl.col("alt").str.contains(r"\|")).select("alt")
print(f"\nALT values with pipes: {alt_with_pipes.height}")
if alt_with_pipes.height > 0:
    print("Examples:")
    print(alt_with_pipes.head(5))


In [ ]:
# Polars code to flatmap pipe-separated ALT alleles into separate rows
import polars as pl

def flatmap_alt_alleles(df: pl.LazyFrame, alt_column: str = "alt", separator: str = "|") -> pl.LazyFrame:
    """
    Flatmap pipe-separated ALT alleles into separate rows.
    
    Args:
        df: Input LazyFrame with pipe-separated ALT alleles
        alt_column: Name of the ALT column (default: "alt")
        separator: Separator used in ALT field (default: "|")
    
    Returns:
        LazyFrame with each ALT allele in a separate row
    """
    return (
        df
        .with_columns(
            # Split the ALT column on the separator
            pl.col(alt_column).str.split(separator).alias(f"{alt_column}_split")
        )
        .explode(f"{alt_column}_split")  # Explode to create separate rows
        .drop(alt_column)  # Remove original ALT column
        .rename({f"{alt_column}_split": alt_column})  # Rename back to original column name
        .filter(pl.col(alt_column).str.len() > 0)  # Remove empty strings
    )

# Example usage with sample data
# Let's create a test DataFrame with pipe-separated ALT alleles
test_data = pl.DataFrame({
    "chrom": ["chr1", "chr1", "chr2", "chr3"],
    "pos": [1000, 2000, 3000, 4000],
    "ref": ["A", "G", "T", "C"],
    "alt": ["T|C", "A", "G|A|C", "T|G"],
    "quality": [30, 25, 40, 35]
})

print("Original data:")
print(test_data)
print("\nAfter flatmapping ALT alleles:")
result = flatmap_alt_alleles(test_data.lazy()).collect()
print(result)


In [ ]:
# Alternative approach: Handle both comma and pipe separators
def flatmap_alt_alleles_flexible(df: pl.LazyFrame, alt_column: str = "alt") -> pl.LazyFrame:
    """
    Flatmap ALT alleles with flexible separator detection (handles both , and |).
    
    Args:
        df: Input LazyFrame with separated ALT alleles
        alt_column: Name of the ALT column (default: "alt")
    
    Returns:
        LazyFrame with each ALT allele in a separate row
    """
    return (
        df
        .with_columns(
            # First replace pipes with commas, then split on commas
            pl.col(alt_column)
            .str.replace_all(r"\|", ",")
            .str.split(",")
            .alias(f"{alt_column}_split")
        )
        .explode(f"{alt_column}_split")
        .drop(alt_column)
        .rename({f"{alt_column}_split": alt_column})
        .filter(pl.col(alt_column).str.len() > 0)  # Remove empty strings
        .with_columns(
            pl.col(alt_column).str.strip_chars()  # Remove any whitespace
        )
    )

# Test with mixed separators
mixed_data = pl.DataFrame({
    "chrom": ["chr1", "chr1", "chr2", "chr3", "chr4"],
    "pos": [1000, 2000, 3000, 4000, 5000],
    "ref": ["A", "G", "T", "C", "A"],
    "alt": ["T|C", "A,G", "G|A|C", "T,G", "C"],
    "quality": [30, 25, 40, 35, 45]
})

print("Mixed separator data:")
print(mixed_data)
print("\nAfter flexible flatmapping:")
flexible_result = flatmap_alt_alleles_flexible(mixed_data.lazy()).collect()
print(flexible_result)


In [ ]:
# Advanced version: Preserve original row info and add allele index
def flatmap_alt_alleles_with_index(df: pl.LazyFrame, alt_column: str = "alt", separator: str = "|") -> pl.LazyFrame:
    """
    Flatmap ALT alleles while preserving original row information and adding allele index.
    
    Args:
        df: Input LazyFrame with separated ALT alleles
        alt_column: Name of the ALT column (default: "alt")
        separator: Separator used in ALT field (default: "|")
    
    Returns:
        LazyFrame with each ALT allele in a separate row, including original_row_id and allele_index
    """
    return (
        df
        .with_row_index("original_row_id")  # Add original row ID
        .with_columns(
            pl.col(alt_column).str.split(separator).alias(f"{alt_column}_split")
        )
        .explode(f"{alt_column}_split")
        .with_columns(
            # Add allele index within each original row
            pl.int_range(pl.len()).over("original_row_id").alias("allele_index")
        )
        .drop(alt_column)
        .rename({f"{alt_column}_split": alt_column})
        .filter(pl.col(alt_column).str.len() > 0)
    )

# Test with the advanced version
print("Advanced version with row ID and allele index:")
advanced_result = flatmap_alt_alleles_with_index(test_data.lazy()).collect()
print(advanced_result)

# Show how many alleles each original row had
allele_counts = (
    advanced_result
    .group_by("original_row_id")
    .agg(
        pl.col("allele_index").max().alias("max_allele_index"),
        pl.col("alt").count().alias("allele_count"),
        pl.col("alt").alias("alleles")
    )
    .sort("original_row_id")
)
print("\nAllele counts per original row:")
print(allele_counts)


In [ ]:
# Practical example: Apply to your actual VCF data
def process_vcf_with_alt_splitting(vcf_lazy_frame: pl.LazyFrame) -> pl.LazyFrame:
    """
    Process a VCF LazyFrame to split pipe-separated ALT alleles.
    Designed to work with genobear VCF reading functions.
    
    Args:
        vcf_lazy_frame: LazyFrame from VCF file (e.g., from gb.read_vcf_file)
    
    Returns:
        LazyFrame with exploded ALT alleles
    """
    # Check if the DataFrame has the expected VCF columns
    schema = vcf_lazy_frame.collect_schema()
    alt_col = None
    
    # Find the ALT column (could be 'alt', 'ALT', etc.)
    for col_name in schema.names():
        if col_name.lower() == 'alt':
            alt_col = col_name
            break
    
    if alt_col is None:
        raise ValueError("No ALT column found in the VCF data")
    
    return flatmap_alt_alleles_flexible(vcf_lazy_frame, alt_column=alt_col)

# Example of how to use with your actual data:
print("Usage examples:")
print("1. Basic flatmapping:")
print("   result = flatmap_alt_alleles(your_vcf_lazy_frame)")
print()
print("2. Flexible separator handling (comma and pipe):")
print("   result = flatmap_alt_alleles_flexible(your_vcf_lazy_frame)")
print()
print("3. With row tracking:")
print("   result = flatmap_alt_alleles_with_index(your_vcf_lazy_frame)")
print()
print("4. For VCF data specifically:")
print("   result = process_vcf_with_alt_splitting(your_vcf_lazy_frame)")

# Real-world usage pattern:
usage_code = '''
# Load VCF file
vcf_data = gb.read_vcf_file("your_file.vcf")

# Split ALT alleles into separate rows
split_data = flatmap_alt_alleles_flexible(vcf_data)

# Collect results
result = split_data.collect()
'''

print(f"\nReal-world usage pattern:\n{usage_code}")
